# TRANSFORMATION
In the transformation phase, several key data preparation steps were applied to both the raw and incremental datasets to improve data quality and readiness for analysis. First, duplicate records were identified and removed to ensure data integrity. Missing values in columns such as customer_name and region were filled with 'Unknown', while numerical fields like quantity and unit_price were imputed using the median and mean, respectively, to preserve the dataset’s structure. The order_date field was converted from string to datetime format to enable time-based analysis. An enrichment step was also applied by creating a new total_price column, calculated as the product of quantity and unit_price. Lastly, unnecessary or irrelevant columns (e.g., comments, notes) were dropped to reduce noise in the dataset. While various structural, cleaning, enrichment, and filtering transformations were performed, categorization (such as age grouping or customer tiering) was not included in this phase. The final cleaned datasets were saved to the transformed/ directory for loading.

In [17]:
# etl_transform.ipynb

import pandas as pd
import os
from IPython.display import display

# Step 1: Load the cleaned datasets from the 'data/' folder
raw_df = pd.read_csv('1. data/raw_data.csv')
incremental_df = pd.read_csv('1. data/incremental_data.csv')

# Create output directory if not exists
os.makedirs('transformed', exist_ok=True)

### ---------------- TRANSFORMATION 1: Remove Duplicates ---------------- ###

What: Remove exact duplicate rows from both datasets.
Why: Duplicate records can lead to double-counting, biased metrics, and inefficient storage. Removing them ensures data accuracy and integrity.
How: Used DataFrame.drop_duplicates() to eliminate rows that are completely identical.


In [ ]:
print("\n--- TRANSFORMATION 1: Remove Duplicate Rows ---")

print("Raw Data - Before:")
display(raw_df.duplicated().value_counts())

raw_df = raw_df.drop_duplicates()
incremental_df = incremental_df.drop_duplicates()

print("Raw Data - After:")
display(raw_df.duplicated().value_counts())




--- TRANSFORMATION 1: Remove Duplicate Rows ---
Raw Data - Before:


False    99
True      1
Name: count, dtype: int64

Raw Data - After:


False    99
Name: count, dtype: int64

# ---------------- HANDLE MISSING VALUES ----------------

What: Address missing values in key columns like customer_name, region, quantity, and unit_price.

Why: Missing values (NaNs) can break calculations, visualizations, and machine learning models. We need to ensure each column is usabl

How:
Categorical fields like customer_name and region were filled with "Unknown".

Numeric fields:
quantity was filled with the median (resistant to outliers).
unit_price was filled with the mean (captures central tendency).
Dates like order_date were converted to datetime, with invalid entries set to Na

In [ ]:

# Convert numeric columns to proper types first
for df in [raw_df, incremental_df]:
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
    df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce')

# Fill missing values
for df in [raw_df, incremental_df]:
    df['customer_name'] = df['customer_name'].fillna('Unknown')
    df['region'] = df['region'].fillna('Unknown')
    df['quantity'] = df['quantity'].fillna(df['quantity'].median())
    df['unit_price'] = df['unit_price'].fillna(df['unit_price'].mean())

# Convert dates
for df in [raw_df, incremental_df]:
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# Check remaining missing values
print("Missing values in raw_df after cleaning:")
display(raw_df.isnull().sum())

print("Missing values in incremental_df after cleaning:")
display(incremental_df.isnull().sum())



Missing values in raw_df after cleaning:


order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       1
region           0
dtype: int64

Missing values in incremental_df after cleaning:


order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64

# ---------------- FILTERING: Drop Irrelevant Columns ----------------

What: Removed non-essential columns such as comments, notes, or unused_code.

Why: These fields were not relevant to the analysis, transformation, or loading phases. Keeping them would increase noise and storage unnecessarily.

How: Used DataFrame.drop() with a filtered list of column names, only dropping those that actually exist.

In [ ]:
print("\n--- TRANSFORMATION: Filtering Columns ---")

# Example: Drop unnecessary columns like 'comments' or 'unused_code'
columns_to_drop = [col for col in ['comments', 'notes', 'unused_code'] if col in raw_df.columns]

for df in [raw_df, incremental_df]:
    df.drop(columns=columns_to_drop, inplace=True)

print(f"Dropped columns (if present): {columns_to_drop}")
print("Remaining columns in raw_df:")
display(raw_df.columns)



--- TRANSFORMATION: Filtering Columns ---
Dropped columns (if present): []
Remaining columns in raw_df:


Index(['order_id', 'customer_name', 'product', 'quantity', 'unit_price',
       'order_date', 'region', 'total_price', 'age', 'age_group'],
      dtype='object')

### ---------------- TRANSFORMATION 3: Convert Dates to Datetime ---------------- ###

What: Standardize the order_date field to datetime format.

Why: Enables accurate time-based analysis such as sales trends, filtering by date, and time grouping.

How: Used pd.to_datetime() with errors='coerce' to gracefully handle invalid date formats by converting them to NaT.

In [ ]:
print("\n--- TRANSFORMATION 3: Convert signup_date to datetime ---")

if 'signup_date' in raw_df.columns:
    print("Before:")
    display(raw_df['signup_date'].head())

    raw_df['signup_date'] = pd.to_datetime(raw_df['signup_date'], errors='coerce')
    incremental_df['signup_date'] = pd.to_datetime(incremental_df['signup_date'], errors='coerce')

    print("After:")
    display(raw_df['signup_date'].head())




--- TRANSFORMATION 3: Convert signup_date to datetime ---


### ---------------- TRANSFORMATION 4: Add Enrichment Column ---------------- ###

What: Add a new column total_price calculated as quantity * unit_price.

Why: This derived metric is useful for revenue analysis, order value insights, and business reporting.

How: Direct column multiplication:
df['total_price'] = df['quantity'] * df['unit_price']

In [ ]:
print("\n--- TRANSFORMATION 4: Add total_price column = quantity * unit_price ---")

if 'quantity' in raw_df.columns and 'unit_price' in raw_df.columns:
    raw_df['total_price'] = raw_df['quantity'] * raw_df['unit_price']
    incremental_df['total_price'] = incremental_df['quantity'] * incremental_df['unit_price']

    print("New column total_price added:")
    display(raw_df[['quantity', 'unit_price', 'total_price']].head())




--- TRANSFORMATION 4: Add total_price column = quantity * unit_price ---
New column total_price added:


,quantity,unit_price,total_price
0,2.0,500.00000,1000.00000
1,2.0,496.09375,992.18750
2,2.0,250.00000,500.00000
3,2.0,750.00000,1500.00000
4,3.0,496.09375,1488.28125


### ---------------- Save the Transformed Files ---------------- ###

What: Store clean and enriched datasets into the transformed/ directory.

Why: Ensures transformed outputs are ready for loading into a database or analytics tool.

How: Used DataFrame.to_csv() to save:


In [ ]:
raw_df.to_csv('2. transformed/transformed_full.csv', index=False)
incremental_df.to_csv('2. transformed/transformed_incremental.csv', index=False)

print("\n Transformed files saved to 'transformed/' folder.")

# View the transformed full dataset
transformed_full = pd.read_csv('2. transformed/transformed_full.csv')
print("Transformed Full Dataset:")
display(transformed_full.head())

# View the transformed incremental dataset
transformed_inc = pd.read_csv('2. transformed/transformed_incremental.csv')
print("Transformed Incremental Dataset:")
display(transformed_inc.head())


 Transformed files saved to 'transformed/' folder.
Transformed Full Dataset:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,age,age_group
0,1,Diana,Tablet,2.0,500.00000,2024-01-20,South,1000.00000,NaN,Unknown
1,2,Eve,Laptop,2.0,496.09375,2024-04-29,North,992.18750,NaN,Unknown
2,3,Charlie,Laptop,2.0,250.00000,2024-01-08,Unknown,500.00000,NaN,Unknown
3,4,Eve,Laptop,2.0,750.00000,2024-01-07,West,1500.00000,NaN,Unknown
4,5,Eve,Tablet,3.0,496.09375,2024-03-07,South,1488.28125,NaN,Unknown


Transformed Incremental Dataset:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,age,age_group
0,101,Alice,Laptop,1.5,900.0,2024-05-09,Central,1350.0,NaN,Unknown
1,102,Unknown,Laptop,1.0,300.0,2024-05-07,Central,300.0,NaN,Unknown
2,103,Unknown,Laptop,1.0,600.0,2024-05-04,Central,600.0,NaN,Unknown
3,104,Unknown,Tablet,1.5,300.0,2024-05-26,Central,450.0,NaN,Unknown
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North,1200.0,NaN,Unknown
